In [2]:
import requests
import pandas as pd
import json



# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
station_df = pd.read_csv('../data/station_df.csv')
station_df

,Unnamed: 0,names,latitude,longitude,slots
0,0,Chilco & Barclay,49.291909,-123.140713,18
1,1,St George & Broadway,49.262321,-123.093060,14
2,2,Britannia Parking Lot,49.275882,-123.071865,14
3,3,Morton & Denman,49.288030,-123.142135,26
4,4,Thornton & National,49.273777,-123.092723,14
...,...,...,...,...,...
253,253,7th & Heather,49.265089,-123.119425,18
254,254,Union & Dunlevy,49.277595,-123.095830,16
255,255,Richards & Helmcken,49.277141,-123.122589,18
256,256,Keefer & Abbott,49.279821,-123.108020,28


In [27]:
#Contact FourSquare API

def contact_four_square(latitude, longitude, categories, radius=1000, limit=50):

    foursquare_endpoint = "https://api.foursquare.com/v3/places/search"

    #Placeholder for FourSquare API while waiting to sort out environmental variable
    FOURSQUARE_API = "fsq3RyWS1BeG7eRK2SWk6+gxeQ2DpBxrxphrRjPBjQTSme0="
    #Authorization headers
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_API
    }


    #Include search paramters
    #Latitude and Longitude string for params
    ll = f"{latitude},{longitude}"
    
    params = {
        'limit': limit,
        'radius': radius,
        'll': ll,
        'categories': categories 
    }

    #Contact API
    foursquare_response = requests.get(foursquare_endpoint, headers=headers, params=params)

    #Check url for endpoint with params
    # print(foursquare_response.url)

    return foursquare_response.json()['results']



In [28]:
foursquare_json = contact_four_square(station_df.iloc[0]['latitude'], station_df.iloc[0]['longitude'], 10047)

#Write response data to file
file_path = '../data/foursquare.json'

with open(file_path, 'w') as json_file:
    json.dump(foursquare_json, json_file, indent=4)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating